Debug pandas update

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import dbcp

from dbcp.helpers import get_sql_engine
from io import StringIO

/app/.local/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [3]:
import dbcp.transform.geocodio as geocodio

In [8]:
from dbcp.transform.helpers import GEOCODER_CACHES

In [9]:
GEOCODER_CACHES.clear_caches()

In [4]:
pd.set_option('display.max_columns', None)

Debug geocodio

In [6]:
import os

In [7]:
os.environ["GEOCODIO_API_KEY"]

'2a0e447cace827260a7e409064a7a0740467ca7'

In [ ]:
df = pd.DataFrame([{"state": "LA", "county": "Random locality name"}])

In [4]:
df = pd.DataFrame([{"state": "XX", "county": "Random locality name"}])

In [6]:
result

,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
0,Nome,city,Nome Census Area


In [5]:
result = geocodio._geocode_locality.func(
        df, state_col="state", locality_col="county"
    )

/app/dbcp/transform/geocodio.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["address"] = batch[locality_col] + ", " + batch[state_col]


In [1]:
import os

In [2]:
os.environ["GEOCODIO_API_KEY"]

'2a0e447cace827260a7e409064a7a0740467ca7'

In [8]:
df = pd.DataFrame([{"state": "LA", "county": "DeSoto Parish"}])

In [9]:
geocodio._geocode_locality.func(
        df, state_col="state", locality_col="county"
    )

/app/dbcp/transform/geocodio.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["address"] = batch[locality_col] + ", " + batch[state_col]


,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
0,Robeline,city,Natchitoches Parish


In [10]:
geocodio._geocode_locality.func(
        zdf, state_col="state", locality_col="county"
    )

/app/dbcp/transform/geocodio.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["address"] = batch[locality_col] + ", " + batch[state_col]


,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
0,Nome,city,Nome Census Area


Read in data warehouse tables

In [6]:
engine = get_sql_engine()

In [4]:
query = """
    WITH
    j40_counties as (
        SELECT
            DISTINCT SUBSTRING("tract_id_fips", 1, 5) as county_id_fips
        FROM data_warehouse.justice40_tracts
    )
    select
        j.county_id_fips as j40_fips,
        c.county_id_fips as c_fips
    from j40_counties as j
    full outer join data_warehouse.county_fips as c
    USING (county_id_fips)
    where j.county_id_fips is null
    or c.county_id_fips is null
    order by j40_fips desc nulls last, c_fips desc nulls last
    """

In [16]:
j40_counties = pd.read_sql("""SELECT
            DISTINCT SUBSTRING("tract_id_fips", 1, 5) as county_id_fips
        FROM data_warehouse.justice40_tracts""", engine)

In [29]:
c = pd.read_sql("""SELECT county_id_fips from data_warehouse.county_fips""", engine)

In [30]:
j40_counties["j_40"] = True
c["census"] = True

In [32]:
j40_counties.merge(c, how="outer", on = "county_id_fips")

,county_id_fips,j_40,census
0,01001,NaN,True
1,01003,NaN,True
2,01005,NaN,True
3,01007,NaN,True
4,01009,NaN,True
...,...,...,...
3607,90119,True,NaN
3608,90135,True,NaN
3609,90138,True,NaN
3610,90158,True,NaN


In [36]:
from google.cloud import bigquery
from dbcp.extract.helpers import cache_gcs_archive_file_locally

def get_bigquery_table_version(dataset_id, table_name, project_id="dbcp-dev-350818"):
    """
    Get the data version of a BigQuery table.

    The dbcp.commands.publish script generates a version number for each data release
    and adds it as a label to the BQ tables.

    Args:
        dataset_id: the BQ dataset ID
        table_name: the name of the table
        project_id: the GCP project id

    Return:
        the current DBCP version number of the requested table
    """
    client = bigquery.Client()

    table_ref = f"{project_id}.{dataset_id}.{table_name}"
    table = client.get_table(table_ref)  # Fetch table metadata

    labels = table.labels  # Get the labels dictionary
    return labels["version"]

table_name = "justice40_tracts"
version = get_bigquery_table_version("data_warehouse", table_name)
uri = f"gs://dgm-outputs/{version}/data_warehouse/{table_name}.parquet"
data_cache = "/app/data/gcp_outputs"

j40_path = cache_gcs_archive_file_locally(uri, data_cache)
old_j40 = pd.read_parquet(j40_path)

In [38]:
old_j40["county_id_fips"] = old_j40.tract_id_fips.str[0:5]

In [40]:
len(old_j40["county_id_fips"].unique())

3234

In [41]:
len(j40_counties)

3294

In [42]:
"01001" in j40_counties.county_id_fips.unique()

False

In [43]:
"01001" in old_j40.county_id_fips.unique()

True

In [44]:
new_j40 = pd.read_parquet("/app/data/output/data_warehouse/justice40_tracts.parquet")

In [45]:
new_j40

,tract_id_fips,black_percent,aian_percent,asian_percent,native_hawaiian_or_pacific_percent,two_or_more_races_percent,white_percent,hispanic_or_latino_percent,other_races_percent,age_under_10_percent,age_10_to_64_percent,age_over_64_percent,n_thresholds_exceeded,n_categories_exceeded,is_disadvantaged_without_considering_neighbors,is_disadvantaged_based_on_neighbors_and_low_income_threshold,is_disadvantaged_due_to_tribal_overlap,is_disadvantaged,tract_area_disadvantaged_percent,disadvantaged_neighbors_percent,population,individuals_below_2x_federal_poverty_line_percentile,individuals_below_2x_federal_poverty_line_percent,is_low_income,is_income_data_imputed,expected_agriculture_loss_rate_is_low_income,expected_agriculture_loss_percentile,expected_agriculture_loss,expected_building_loss_rate_is_low_income,expected_building_loss_percentile,expected_building_loss,expected_population_loss_rate_is_low_income,expected_population_loss_percentile,expected_population_loss,props_30year_flood_risk_percentile,props_30year_flood_risk_percent,is_props_30year_flood_risk,is_props_30year_flood_risk_is_low_income,props_30year_fire_risk_percentile,props_30year_fire_risk_percent,is_props_30year_fire_risk_percent,is_props_30year_fire_risk_percent_is_low_income,energy_burden_is_low_income,energy_burden_percentile,energy_burden,pm2_5_is_low_income,pm2_5_percentile,pm2_5,diesel_particulates_is_low_income,diesel_particulates_percentile,diesel_particulates,traffic_proximity_is_low_income,traffic_percentile,traffic,dot_transit_barriers_is_low_income,dot_travel_barriers_score_percentile,housing_burden_is_low_income,housing_burden_percentile,housing_burden_percent,lead_paint_and_median_house_value_is_low_income,lead_paint_houses_percentile,lead_paint_houses_percent,median_home_price_percentile,median_home_price,tract_area_covered_by_impervious_surface_is_low_income,tract_area_covered_by_impervious_surface,tract_area_covered_by_impervious_surface_percent,tract_area_covered_by_impervious_surface_percentile,has_35_acres,experienced_historic_underinvestment_and_remains_low_income,experienced_historic_underinvestment,homes_with_no_kitchen_or_indoor_plumbing_percentile,homes_with_no_kitchen_or_indoor_plumbing_percent,proximity_to_hazardous_waste_facilities_is_low_income,hazardous_waste_proximity_percentile,hazardous_waste_proximity,proximity_to_superfund_sites_is_low_income,superfund_proximity_percentile,superfund_proximity,proximity_to_RMP_sites_is_low_income,risk_management_plan_proximity_percentile,risk_management_plan_proximity,has_one_FUDS,has_one_abandoned_mine,has_one_abandoned_mine_is_low_income,has_one_FUDS_is_low_income,has_one_FUDS_missing_data_treated_as_False,has_one_abandoned_mine_missing_data_treated_as_False,wastewater_discharge_is_low_income,wastewater_percentile,wastewater,leaky_underground_storage_tanks_is_low_income,leaky_underground_storage_tanks_percentile,leaky_underground_storage_tanks,asthma_is_low_income,asthma_percentile,asthma,diabetes_is_low_income,diabetes_percentile,diabetes,heart_disease_is_low_income,heart_disease_percentile,heart_disease,low_life_expectancy_is_low_income,life_expectancy_percentile,life_expectancy,low_median_household_income_and_low_hs_attainment,local_to_area_income_ratio_percentile,local_to_area_income_ratio,households_in_linguistic_isolation_and_low_hs_attainment,linguistic_isolation_percentile,linguistic_isolation_percent,unemployment_and_low_hs_attainment,unemployment_percentile,unemployment_percent,households_below_federal_poverty_level_low_hs_attainment,below_2x_poverty_line_percentile,below_2x_poverty_line_percent,below_poverty_line_percentile,below_poverty_line_percent,less_than_high_school_percentile,less_than_high_school_percent,non_college_students_percent,unemployment_2010_percent,below_poverty_line_2010_percent,unemployment_and_low_hs_edu_islands,households_below_federal_poverty_level_low_hs_edu_islands,low_median_household_income_and_low_hs_edu_islands,number_of_tribal_areas_within_tract_for_alaska,

In [46]:
old_j40

,tract_id_fips,black_percent,aian_percent,asian_percent,native_hawaiian_or_pacific_percent,two_or_more_races_percent,white_percent,hispanic_or_latino_percent,other_races_percent,age_under_10_percent,age_10_to_64_percent,age_over_64_percent,n_thresholds_exceeded,n_categories_exceeded,is_disadvantaged_without_considering_neighbors,is_disadvantaged_based_on_neighbors_and_low_income_threshold,is_disadvantaged_due_to_tribal_overlap,is_disadvantaged,tract_area_disadvantaged_percent,disadvantaged_neighbors_percent,population,individuals_below_2x_federal_poverty_line_percentile,individuals_below_2x_federal_poverty_line_percent,is_low_income,is_income_data_imputed,expected_agriculture_loss_rate_is_low_income,expected_agriculture_loss_percentile,expected_agriculture_loss,expected_building_loss_rate_is_low_income,expected_building_loss_percentile,expected_building_loss,expected_population_loss_rate_is_low_income,expected_population_loss_percentile,expected_population_loss,props_30year_flood_risk_percentile,props_30year_flood_risk_percent,is_props_30year_flood_risk,is_props_30year_flood_risk_is_low_income,props_30year_fire_risk_percentile,props_30year_fire_risk_percent,is_props_30year_fire_risk_percent,is_props_30year_fire_risk_percent_is_low_income,energy_burden_is_low_income,energy_burden_percentile,energy_burden,pm2_5_is_low_income,pm2_5_percentile,pm2_5,diesel_particulates_is_low_income,diesel_particulates_percentile,diesel_particulates,traffic_proximity_is_low_income,traffic_percentile,traffic,dot_transit_barriers_is_low_income,dot_travel_barriers_score_percentile,housing_burden_is_low_income,housing_burden_percentile,housing_burden_percent,lead_paint_and_median_house_value_is_low_income,lead_paint_houses_percentile,lead_paint_houses_percent,median_home_price_percentile,median_home_price,tract_area_covered_by_impervious_surface_is_low_income,tract_area_covered_by_impervious_surface,tract_area_covered_by_impervious_surface_percent,tract_area_covered_by_impervious_surface_percentile,has_35_acres,experienced_historic_underinvestment_and_remains_low_income,experienced_historic_underinvestment,homes_with_no_kitchen_or_indoor_plumbing_percentile,homes_with_no_kitchen_or_indoor_plumbing_percent,proximity_to_hazardous_waste_facilities_is_low_income,hazardous_waste_proximity_percentile,hazardous_waste_proximity,proximity_to_superfund_sites_is_low_income,superfund_proximity_percentile,superfund_proximity,proximity_to_RMP_sites_is_low_income,risk_management_plan_proximity_percentile,risk_management_plan_proximity,has_one_FUDS,has_one_abandoned_mine,has_one_abandoned_mine_is_low_income,has_one_FUDS_is_low_income,has_one_FUDS_missing_data_treated_as_False,has_one_abandoned_mine_missing_data_treated_as_False,wastewater_discharge_is_low_income,wastewater_percentile,wastewater,leaky_underground_storage_tanks_is_low_income,leaky_underground_storage_tanks_percentile,leaky_underground_storage_tanks,asthma_is_low_income,asthma_percentile,asthma,diabetes_is_low_income,diabetes_percentile,diabetes,heart_disease_is_low_income,heart_disease_percentile,heart_disease,low_life_expectancy_is_low_income,life_expectancy_percentile,life_expectancy,low_median_household_income_and_low_hs_attainment,local_to_area_income_ratio_percentile,local_to_area_income_ratio,households_in_linguistic_isolation_and_low_hs_attainment,linguistic_isolation_percentile,linguistic_isolation_percent,unemployment_and_low_hs_attainment,unemployment_percentile,unemployment_percent,households_below_federal_poverty_level_low_hs_attainment,below_2x_poverty_line_percentile,below_2x_poverty_line_percent,below_poverty_line_percentile,below_poverty_line_percent,less_than_high_school_percentile,less_than_high_school_percent,non_college_students_percent,unemployment_2010_percent,below_poverty_line_2010_percent,unemployment_and_low_hs_edu_islands,households_below_federal_poverty_level_low_hs_edu_islands,low_median_household_income_and_low_hs_edu_islands,number_of_tribal_areas_within_tract_for_alaska,

In [7]:
actual = pd.read_sql(query, engine).astype("string")

In [8]:
actual

,j40_fips,c_fips
0,90159,<NA>
1,90158,<NA>
2,90138,<NA>
3,90135,<NA>
4,90119,<NA>
...,...,...
691,<NA>,01009
692,<NA>,01007
693,<NA>,01005
694,<NA>,01003


In [14]:
expected = pd.read_csv(
        StringIO(
            """j40_fips,c_fips
51515,null
46113,null
02270,null
02261,null
null,46102
null,02158
null,02066
null,02063
"""))

In [15]:
expected

,j40_fips,c_fips
0,51515.0,NaN
1,46113.0,NaN
2,2270.0,NaN
3,2261.0,NaN
4,NaN,46102.0
5,NaN,2158.0
6,NaN,2066.0
7,NaN,2063.0
